# Etapa 3 – Webscraping de Fontes Complementares
Esta etapa tem como objetivo identificar e coletar conteúdos técnicos relevantes disponíveis online que não estejam presentes nos documentos em PDF já processados no projeto. A inclusão dessas fontes complementares visa enriquecer a base textual do Agente de Inteligência Artificial com informações mais recentes, específicas e operacionais, oriundas de instituições confiáveis.

**3.1 Levantamento de Fontes**
Inicialmente, foram mapeadas páginas institucionais estratégicas que frequentemente publicam documentos técnicos e científicos relevantes para a saúde pública. As fontes priorizadas incluem:

- OPAS/OMS (Organização Pan-Americana da Saúde)

- Ministério da Saúde (Brasil)

- Anvisa

- Fiocruz

- CDC (Centers for Disease Control and Prevention – EUA)

A partir dessas páginas, foram identificadas seções com documentos atualizados, publicações técnicas, boletins informativos, relatórios de monitoramento e notas normativas. Esses materiais abrangem temas como imunização, doenças transmissíveis e não transmissíveis, saúde mental, vigilância laboratorial, práticas clínicas, vetores e segurança alimentar, entre outros.

As informações levantadas foram organizadas em uma planilha com os seguintes campos:

- Fonte

- Link

- Tipo de conteúdo

- Doença/Tema (resumido)

- Observações

**3.2 Coleta dos Conteúdos**
Para automatizar a coleta dos títulos e links das publicações mais recentes, foi desenvolvido um script em Python com uso da biblioteca BeautifulSoup, capaz de varrer automaticamente os sites indicados e extrair os principais conteúdos disponíveis. O script realiza:

- Acesso às páginas institucionais listadas;

- Extração dos títulos e links de documentos técnicos visíveis;

- Organização dos resultados em uma planilha .csv para posterior análise e validação manual.

*Essa abordagem permite manter a base textual atualizada com agilidade, além de facilitar a inclusão sistemática de novos documentos à medida que forem sendo publicados.*

## **3.1 Levantamento de Fontes**
Inicialmente, foram mapeadas páginas institucionais estratégicas que frequentemente publicam documentos técnicos e científicos relevantes para a saúde pública. As fontes priorizadas incluem:

- OPAS/OMS (Organização Pan-Americana da Saúde)

- Ministério da Saúde (Brasil)

- Anvisa

- Fiocruz

- CDC (Centers for Disease Control and Prevention – EUA)

A partir dessas páginas, foram identificadas seções com documentos atualizados, publicações técnicas, boletins informativos, relatórios de monitoramento e notas normativas. Esses materiais abrangem temas como imunização, doenças transmissíveis e não transmissíveis, saúde mental, vigilância laboratorial, práticas clínicas, vetores e segurança alimentar, entre outros.

As informações levantadas foram organizadas em uma planilha com os seguintes campos:

- Fonte

- Link

- Tipo de conteúdo

- Doença/Tema (resumido)

- Observações

In [6]:
#preparando o ambiente
# Instalação das bibliotecas necessárias
%pip install requests beautifulsoup4 pandas
%pip install pandas beautifulsoup4 python-slugify openpyxl
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
from slugify import slugify

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Etapa de Webscraping – Coleta de Fontes Complementares

Nesta etapa, foi realizada uma coleta automatizada de conteúdos a partir de páginas institucionais selecionadas como fontes complementares de informação técnica, científica e normativa. Foram incluídos portais da OPAS, Ministério da Saúde, Anvisa, Fiocruz e CDC.

O script acessa cada site da lista definida, extrai os links disponíveis com títulos legíveis e monta uma tabela com os principais resultados. Apenas links com títulos minimamente descritivos (mais de 10 caracteres) e com URLs completas são considerados. Para cada site, são coletados até 10 links relevantes.

O resultado é salvo em um arquivo CSV chamado `webscraping_fontes_complementares.csv`, contendo três colunas: `Fonte`, `Título` e `Link`. Esse arquivo será utilizado nas etapas seguintes para filtragem e classificação automática dos conteúdos coletados.

In [3]:
# Lista de fontes
sites = [
    ("OPAS - Documentos Técnicos", "https://www.paho.org/pt/documentos-tecnicos-e-cientificos"),
    ("OPAS - Notícias Técnicas", "https://www.paho.org/pt/noticias/noticias-das-unidades-tecnicas"),
    ("OPAS - Publicações", "https://www.paho.org/pt/publicacoes"),
    ("Ministério da Saúde - Saúde A-Z", "https://www.gov.br/saude/pt-br/assuntos/saude-de-a-a-z"),
    ("Anvisa - Notícias", "https://www.gov.br/anvisa/pt-br/assuntos/noticias-anvisa"),
    ("Fiocruz - Notas Técnicas", "https://portal.fiocruz.br/pesquisas-notas-tecnicas-e-relatorios"),
    ("CDC - A-Z Topics", "https://www.cdc.gov/health-topics.html#cdc-atozlist")
]

# Função para extrair os links com títulos legíveis
def extrair_links(nome_fonte, url):
    try:
        resp = requests.get(url, timeout=15)
        soup = BeautifulSoup(resp.text, 'html.parser')
        links = soup.find_all('a', href=True)
        
        resultados = []
        for tag in links:
            texto = tag.get_text(strip=True)
            href = tag['href']
            if texto and len(texto) > 10 and ('http' in href or href.startswith('/')):
                link_completo = href if 'http' in href else url.split('/pt')[0] + href
                resultados.append({'Fonte': nome_fonte, 'Título': texto, 'Link': link_completo})
        
        return resultados[:10]  # Limita para 10 primeiros por site
    except Exception as e:
        return [{'Fonte': nome_fonte, 'Título': f'Erro ao acessar: {e}', 'Link': url}]

# Rodar scraping
todos_resultados = []
for nome, link in sites:
    todos_resultados.extend(extrair_links(nome, link))

# Criar DataFrame
df = pd.DataFrame(todos_resultados)

# Salvar em CSV
df.to_csv("webscraping_fontes_complementares.csv", index=False, encoding='utf-8-sig')

print("✅ Finalizado. Arquivo 'webscraping_fontes_complementares.csv' salvo com os resultados.")


✅ Finalizado. Arquivo 'webscraping_fontes_complementares.csv' salvo com os resultados.


###  Limpeza e Classificação das Fontes Complementares

Após a coleta inicial dos links, esta etapa realiza a limpeza e a classificação automática dos conteúdos extraídos. As ações realizadas incluem:

1. **Remoção de registros irrelevantes**:
   - Títulos vazios ou muito curtos (menos de 10 caracteres) são eliminados.
   - Links duplicados são removidos para evitar redundâncias.
   - Termos genéricos ou institucionais (ex: "fale conosco", "barra de governo") são filtrados para manter apenas conteúdos técnicos relevantes.

2. **Classificação automatizada**:
   - Cada item é classificado por tipo de conteúdo: *Notícia Técnica*, *Relatório Técnico*, *Publicação Técnica*, *Portal de Publicações* ou *Outro*.
   - Também é feita uma classificação temática com base em palavras-chave, agrupando os conteúdos por áreas como *Imunização*, *Doenças Crônicas*, *Agrotóxicos*, entre outras.

3. **Organização e exportação**:
   - As colunas são reorganizadas para melhor visualização.
   - O resultado final é salvo como `fontes_complementares_classificado.xlsx`, que servirá de base para etapas posteriores de análise ou curadoria manual.


In [4]:
# Caminho para o seu arquivo CSV gerado anteriormente
arquivo_entrada = r'C:\Users\isisi\Documents\IAVS_PROJETO\webscraping_fontes_complementares.csv'
arquivo_saida = r'C:\Users\isisi\Documents\IAVS_PROJETO\fontes_complementares_classificado.xlsx'

# Carregar o CSV
df = pd.read_csv(arquivo_entrada)

# 1. Remover títulos vazios ou muito curtos
df = df[df["Título"].str.len() > 10]

# 2. Remover duplicatas com base no link
df = df.drop_duplicates(subset="Link")

# 3. Remover registros genéricos
palavras_excluir = [
    "inicial", "barra de governo", "fale conosco", "visite", "acessibilidade",
    "linha do tempo", "galeria de", "curiosidades", "cdc.gov home", "other languages",
    "imposto de renda", "atualize"
]
df = df[~df["Título"].str.lower().str.contains('|'.join(palavras_excluir))]

# 4. Classificação automática
def classificar_tipo(texto):
    texto = texto.lower()
    if "notícia" in texto or "news" in texto:
        return "Notícia Técnica"
    elif "relatório" in texto or "report" in texto:
        return "Relatório Técnico"
    elif "manual" in texto or "guia" in texto or "documento" in texto:
        return "Publicação Técnica"
    elif "publicações" in texto or "publications" in texto:
        return "Portal de Publicações"
    else:
        return "Outro"

def classificar_tema(texto):
    texto = texto.lower()
    if "agrotóxico" in texto or "resíduos" in texto:
        return "Agrotóxicos"
    elif "vacina" in texto or "imunização" in texto:
        return "Imunização"
    elif "cardiovascular" in texto or "hearts" in texto:
        return "Doenças Crônicas"
    elif "envelhecimento" in texto:
        return "Envelhecimento"
    elif "migração" in texto:
        return "Saúde de Populações Vulneráveis"
    elif "trânsito" in texto or "acidentes" in texto:
        return "Acidentes e Violência"
    else:
        return "Saúde Pública Geral"

df["Tipo de Conteúdo"] = df["Título"].apply(classificar_tipo)
df["Doença/Tema"] = df["Título"].apply(classificar_tema)
df["Observações"] = "Fonte complementar automatizada – não presente nos PDFs"

# Reordenar colunas
df_final = df[["Fonte", "Título", "Link", "Tipo de Conteúdo", "Doença/Tema", "Observações"]]

# Salvar em Excel
df_final.to_excel(arquivo_saida, index=False)

print("✅ Planilha 'fontes_complementares_classificado.xlsx' gerada com sucesso!")


✅ Planilha 'fontes_complementares_classificado.xlsx' gerada com sucesso!


### Etapa de Extração e Organização dos Textos Complementares

**Objetivo:**  
Extrair o conteúdo textual das páginas web listadas na etapa anterior de webscraping, convertendo automaticamente os textos em arquivos `.txt`, organizados em uma estrutura padrão. Esta etapa tem como finalidade alimentar o pipeline do projeto IAVS com textos limpos, classificados e organizados por fonte.

**Estrutura esperada:**  
- Uma planilha `.xlsx` com as colunas: `Fonte`, `Título`, `Link`, `Tipo de Conteúdo`, `Doença/Tema`, `Observações`.

**O que foi feito:**  
- Leitura da planilha de links coletados e classificados.
- Remoção de registros duplicados e conteúdos irrelevantes.
- Acesso automático a cada link para extração do conteúdo textual.
- Conversão dos textos extraídos em arquivos `.txt`.
- Limpeza dos textos (remoção de artefatos, espaços extras, etc.).
- Organização dos arquivos em duas pastas:  
  - `textos_complementares_raw`: versão bruta extraída.  
  - `textos_complementares_limpos`: versão limpa e pronta para uso.
- Criação automática de todas as pastas necessárias, caso ainda não existam.



In [8]:
# === CONFIGURAÇÃO GERAL ===
arquivo_excel = r'C:\Users\isisi\Documents\IAVS_PROJETO\fontes_complementares_classificado.xlsx'
pasta_raw = r'C:\Users\isisi\Documents\IAVS_PROJETO\textos_complementares_raw'
pasta_limpos = r'C:\Users\isisi\Documents\IAVS_PROJETO\textos_convertidos'
pasta_complementares_limpos = r'C:\Users\isisi\Documents\IAVS_PROJETO\textos_complementares_limpos'
# === Funções auxiliares ===

def criar_pasta(pasta):
    if not os.path.exists(pasta):
        os.makedirs(pasta)

def classificar_tipo(texto):
    texto = texto.lower()
    if "notícia" in texto or "news" in texto:
        return "Notícia Técnica"
    elif "relatório" in texto or "report" in texto:
        return "Relatório Técnico"
    elif "manual" in texto or "guia" in texto or "documento" in texto:
        return "Publicação Técnica"
    elif "publicações" in texto or "publications" in texto:
        return "Portal de Publicações"
    else:
        return "Outro"

def classificar_tema(texto):
    texto = texto.lower()
    if "agrotóxico" in texto or "resíduos" in texto:
        return "Agrotóxicos"
    elif "vacina" in texto or "imunização" in texto:
        return "Imunização"
    elif "cardiovascular" in texto or "hearts" in texto:
        return "Doenças Crônicas"
    elif "envelhecimento" in texto:
        return "Envelhecimento"
    elif "migração" in texto:
        return "Populações Vulneráveis"
    elif "trânsito" in texto or "acidentes" in texto:
        return "Acidentes e Violência"
    else:
        return "Saúde Pública Geral"

def limpar_texto(texto):
    texto = re.sub(r'\n+', '\n', texto)
    texto = re.sub(r'\s+', ' ', texto)
    return texto.strip()

# === Etapa 1: Leitura e classificação da planilha ===
print("📊 Lendo e classificando registros...")
df = pd.read_excel(arquivo_excel)

df = df[df["Título"].str.len() > 10]
df = df.drop_duplicates(subset="Link")

palavras_excluir = [
    "inicial", "barra de governo", "fale conosco", "visite", "acessibilidade",
    "linha do tempo", "galeria de", "curiosidades", "cdc.gov home", "other languages",
    "imposto de renda", "atualize"
]
df = df[~df["Título"].str.lower().str.contains('|'.join(palavras_excluir))]

df["Tipo de Conteúdo"] = df["Título"].apply(classificar_tipo)
df["Doença/Tema"] = df["Título"].apply(classificar_tema)
df["Observações"] = "Fonte complementar automatizada – não presente nos PDFs"
df = df[["Fonte", "Título", "Link", "Tipo de Conteúdo", "Doença/Tema", "Observações"]]

# === Etapa 2: Download dos textos ===
print("Iniciando download dos textos...")
criar_pasta(pasta_raw)
criar_pasta(pasta_complementares_limpos)

for _, row in df.iterrows():
    fonte = slugify(row["Fonte"])
    titulo = slugify(row["Título"])[:60]
    url = row["Link"]
    
    pasta_fonte = os.path.join(pasta_raw, fonte)
    criar_pasta(pasta_fonte)
    
    caminho_saida = os.path.join(pasta_complementares_limpos, f"{titulo}.txt")
    caminho_saida2 = os.path.join(pasta_limpos, f"{titulo}.txt")
    
    try:
        res = requests.get(url, timeout=15)
        soup = BeautifulSoup(res.text, "html.parser")
        texto = "\n".join([tag.get_text(strip=True) for tag in soup.find_all(["p", "h1", "h2", "h3", "li"])])
        
        if len(texto) > 100:
            with open(caminho_saida, "w", encoding="utf-8") as f:
                f.write(texto)
            print(f"✅ Texto salvo: {titulo}")
        else:
            print(f"⚠️ Conteúdo insuficiente: {titulo}")
    except Exception as e:
        print(f"❌ Erro em {url}: {e}")

# === Etapa 3: Limpeza dos textos ===
print("\nLimpando textos e salvando versão final...")
criar_pasta(pasta_limpos)

for fonte in os.listdir(pasta_raw):
    pasta_fonte_raw = os.path.join(pasta_raw, fonte)
    pasta_fonte_limpos = os.path.join(pasta_limpos, fonte)
    criar_pasta(pasta_fonte_limpos)

    for arquivo in os.listdir(pasta_fonte_raw):
        if not arquivo.endswith(".txt"):
            continue

        caminho_entrada = os.path.join(pasta_fonte_raw, arquivo)
        caminho_saida = os.path.join(pasta_fonte_limpos, arquivo)

        try:
            with open(caminho_entrada, 'r', encoding='utf-8') as f:
                conteudo = f.read()

            texto_limpo = limpar_texto(conteudo)

            if len(texto_limpo) > 100:
                with open(caminho_saida, 'w', encoding='utf-8') as f:
                    f.write(texto_limpo)
                print(f"✅ Texto limpo: {arquivo}")
            else:
                print(f"⚠️ Ignorado (muito curto): {arquivo}")
        except Exception as e:
            print(f"❌ Erro ao limpar {arquivo}: {e}")

print("\nPipeline finalizado com sucesso!")
print(f"✔️ Arquivos crus: {pasta_raw}")
print(f"✔️ Arquivos limpos: {pasta_limpos}")


📊 Lendo e classificando registros...
Iniciando download dos textos...
✅ Texto salvo: publicacoes
✅ Texto salvo: biblioteca-digital-de-saude
⚠️ Conteúdo insuficiente: biblioteca-virtual-em-saude
✅ Texto salvo: campus-virtual-de-saude-publica
✅ Texto salvo: centros-colaboradores-da-opas-oms
✅ Texto salvo: fundos-rotatorio-regionais
✅ Texto salvo: decada-do-envelhecimento-saudavel
✅ Texto salvo: iniciativa-de-eliminacao
✅ Texto salvo: migracao-e-saude-nas-americas
✅ Texto salvo: hearts-nas-americas
✅ Texto salvo: ministerio-da-saude
✅ Texto salvo: noticias-para-os-estados
✅ Texto salvo: distrito-federal
✅ Texto salvo: espirito-santo
✅ Texto salvo: mato-grosso
✅ Texto salvo: mato-grosso-do-sul
✅ Texto salvo: minas-gerais
✅ Texto salvo: agencia-nacional-de-vigilancia-sanitaria-anvisa
✅ Texto salvo: agrotoxicos
✅ Texto salvo: disque-intoxicacao
✅ Texto salvo: monografias-de-agrotoxicos
✅ Texto salvo: programa-de-analise-de-residuos-de-agrotoxicos-em-alimentos-
⚠️ Conteúdo insuficiente: reava